# BERT model

# STRUCTURE
1. Load the training data.
2. Adapt data to BERT
3. Obtain a pretrained BERT and fine-tune.
4. Evaluate BERT results.

## 1. Load the training data

In [5]:
import pickle
from sklearn.model_selection import train_test_split
import os

In [6]:
DIRECTORY_NAME = input("Specify the directory you wish to save ALL the data to: ")
os.makedirs(DIRECTORY_NAME)

Specify the directory you wish to save ALL the data to: BERT_results_final


FileExistsError: [Errno 17] File exists: 'BERT_results_final'

In [7]:
def load_pickle_data(filename):
    with open(filename, "rb") as load_file:
        return pickle.load(load_file)

In [9]:
X_data_1 = load_pickle_data("/home/lovhag/storage/data/BERT_data_final/with_eval_X_data_1_train.pickle")
X_data_2 = load_pickle_data("/home/lovhag/storage/data/BERT_data_final/with_eval_X_data_2_train.pickle")
y_data = load_pickle_data("/home/lovhag/storage/data/BERT_data_final/with_eval_y_data_train.pickle")

In [10]:
print(f"Number of data samples: {len(y_data)}")

Number of data samples: 121678


In [11]:
X_data_1[1]

'( vii ) BGL - General Trust Fund for the Core Programme Budget for the Biosafety Protocol , which is extended through 31 December 2011 ; ( viii ) BHL - Special Voluntary Trust Fund for Additional Voluntary Contributions in Support of Approved Activities of the Biosafety Protocol , which is extended through 31 December 2011 ; ( ix ) BTL - General Trust Fund for the Conservation of European Bats ( EUROBATS ) , which is extended through 31 December 2014 ;'

In [12]:
X_data_2[1]

'Following consultation in 2007 , the Government is considering replacing the current legislation with a single Equality Act . As part of this , it is considering the case for extending protection from age discrimination outside the workplace and for extending positive duties on public authorities to the other protected grounds . In Northern Ireland , additional protections have been established to promote equality .'

## 2. Adapt data to BERT

In [13]:
import torch
from transformers import BertTokenizer

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(X_data_1, X_data_2, truncation=True, padding=True)

In [24]:
train_encodings_for_prediction = tokenizer(X_data_1, X_data_2, truncation=True, padding=True, return_tensors='pt')

In [17]:
class SenseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SenseDataset(train_encodings, y_data)

In [26]:
len(train_dataset)

121678

In [28]:
train_dataset[0]

{'input_ids': tensor([  101,  1006,  8890,  1007,  1038, 23296,  1011,  2236,  3404,  4636,
          2005,  1996,  4563,  4746,  5166,  2005,  1996, 16012,  3736,  7959,
          3723,  8778,  1010,  2029,  2003,  3668,  2083,  2861,  2285,  2249,
          1025,  1006,  9937,  1007,  1038,  7317,  1011,  2569, 10758,  3404,
          4636,  2005,  3176, 10758,  5857,  1999,  2490,  1997,  4844,  3450,
          1997,  1996, 16012,  3736,  7959,  3723,  8778,  1010,  2029,  2003,
          3668,  2083,  2861,  2285,  2249,  1025,  1006, 11814,  1007, 18411,
          2140,  1011,  2236,  3404,  4636,  2005,  1996,  5680,  1997,  2647,
         12236,  1006,  9944, 14479,  2015,  1007,  1010,  2029,  2003,  3668,
          2083,  2861,  2285,  2297,  1025,   102,  1996,  2372,  1997,  1996,
          2473,  2036,  3825,  1037,  7050,  2000, 22135, 17922, 28516,  2050,
          2005,  2010,  2590,  2535,  1998,  6691,  2004,  1996,  2569,  4387,
          1997,  1996,  3187,  1011,  2

In [14]:
def save_data_with_pickle(data_dict, folder_name=None):
    if not folder_name:
        folder_name = input(f"Specify which prefix filename you wish to save {list(data_dict.keys())} to: ")
    if folder_name:
        for key, value in data_dict.items():
            filename = folder_name+"/"+key+".pickle"
            with open(filename, "wb") as fp:   #Pickling
                pickle.dump(value, fp)

In [15]:
save_datasets_to_folder = input("Which folder should the datasets be saved to?: ")
save_data_with_pickle({"train_dataset": train_dataset, "val_dataset": val_dataset}, save_datasets_to_folder)

Which folder should the datasets be saved to?: /home/lovhag/storage/data/BERT_data_1


## 3. Obtain a pretrained BERT and fine-tune

In [18]:
import pickle
import torch

In [7]:
class SenseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
def load_pickle_data(filename):
    with open(filename, "rb") as load_file:
        return pickle.load(load_file)

train_dataset = load_pickle_data("/home/lovhag/storage/data/BERT_data_1"+"/train_dataset.pickle")

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Using', device)

Using cuda


In [29]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# keep the weights of the pre-trained encoder frozen and optimize only the weights of the head layers
for param in model.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir=DIRECTORY_NAME+'/BERT_results_final',          # output directory
    num_train_epochs=4,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=DIRECTORY_NAME+'/BERT_logs_final',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

trainer.train()

trainer.save_model(DIRECTORY_NAME+'/BERT_model_final')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'loss': 0.6875601806640625, 'learning_rate': 5e-05, 'epoch': 0.2628811777076761, 'total_flos': 10762693312512000, 'step': 500}
{'loss': 0.671542236328125, 'learning_rate': 4.648283624085538e-05, 'epoch': 0.5257623554153522, 'total_flos': 21525386625024000, 'step': 1000}
{'loss': 0.6713079833984374, 'learning_rate': 4.296567248171075e-05, 'epoch': 0.7886435331230284, 'total_flos': 32288079937536000, 'step': 1500}



{'loss': 0.6677021484375, 'learning_rate': 3.9448508722566127e-05, 'epoch': 1.0515247108307044, 'total_flos': 43033956541747200, 'step': 2000}
{'loss': 0.66580810546875, 'learning_rate': 3.59313449634215e-05, 'epoch': 1.3144058885383807, 'total_flos': 53796649854259200, 'step': 2500}
{'loss': 0.66675439453125, 'learning_rate': 3.241418120427687e-05, 'epoch': 1.5772870662460567, 'total_flos': 64559343166771200, 'step': 3000}
{'loss': 0.665990478515625, 'learning_rate': 2.8897017445132247e-05, 'epoch': 1.840168243953733, 'total_flos': 75322036479283200, 'step': 3500}



{'loss': 0.66303662109375, 'learning_rate': 2.537985368598762e-05, 'epoch': 2.103049421661409, 'total_flos': 86067913083494400, 'step': 4000}
{'loss': 0.6658017578125, 'learning_rate': 2.1862689926842993e-05, 'epoch': 2.365930599369085, 'total_flos': 96830606396006400, 'step': 4500}
{'loss': 0.66045068359375, 'learning_rate': 1.8345526167698368e-05, 'epoch': 2.6288117770767614, 'total_flos': 107593299708518400, 'step': 5000}
{'loss': 0.66278076171875, 'learning_rate': 1.4828362408553741e-05, 'epoch': 2.891692954784437, 'total_flos': 118355993021030400, 'step': 5500}



{'loss': 0.66191162109375, 'learning_rate': 1.1311198649409118e-05, 'epoch': 3.1545741324921135, 'total_flos': 129101869625241600, 'step': 6000}
{'loss': 0.66073193359375, 'learning_rate': 7.79403489026449e-06, 'epoch': 3.4174553101997898, 'total_flos': 139864562937753600, 'step': 6500}
{'loss': 0.6628544921875, 'learning_rate': 4.276871131119865e-06, 'epoch': 3.680336487907466, 'total_flos': 150627256250265600, 'step': 7000}
{'loss': 0.6613369140625, 'learning_rate': 7.597073719752392e-07, 'epoch': 3.943217665615142, 'total_flos': 161389949562777600, 'step': 7500}




In [32]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# keep the weights of the pre-trained encoder frozen and optimize only the weights of the head layers
for param in model.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir=DIRECTORY_NAME+'/BERT_results_final_larger_lr',          # output directory
    num_train_epochs=4,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    learning_rate=0.001,
    logging_dir=DIRECTORY_NAME+'/BERT_logs_final_larger_lr',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

trainer.train()

trainer.save_model(DIRECTORY_NAME+'/BERT_model_final_larger_lr')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: 

In [13]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

pt_model = BertForSequenceClassification.from_pretrained("/home/lovhag/projects/dl4nlp_assignment_1/BERT_results/BERT_model_1")

# keep the weights of the pre-trained encoder frozen and optimize only the weights of the head layers
for param in pt_model.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir=DIRECTORY_NAME+'/BERT_results_pt_1',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=DIRECTORY_NAME+'/BERT_logs_pt_1',            # directory for storing logs
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=pt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)

trainer.train()

trainer.save_model(DIRECTORY_NAME+'/BERT_model_pt_1')

{'loss': 0.6672554931640625, 'learning_rate': 3.6855941114616195e-05, 'epoch': 0.2628811777076761, 'total_flos': 10762693312512000, 'step': 500}
{'loss': 0.665552490234375, 'learning_rate': 2.3711882229232387e-05, 'epoch': 0.5257623554153522, 'total_flos': 21525386625024000, 'step': 1000}
{'loss': 0.664870361328125, 'learning_rate': 1.056782334384858e-05, 'epoch': 0.7886435331230284, 'total_flos': 32288079937536000, 'step': 1500}




{'eval_loss': 0.6630513711166883, 'epoch': 1.0, 'total_flos': 40924468652494848, 'step': 1902}



In [25]:
from transformers import BertForNextSentencePrediction, Trainer, TrainingArguments

model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased', return_dict=True)

# keep the weights of the pre-trained encoder frozen and optimize only the weights of the head layers
for param in model.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir=DIRECTORY_NAME+'/BERT_results_final',          # output directory
    num_train_epochs=4,              # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=DIRECTORY_NAME+'/BERT_logs_final',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

trainer.train()

trainer.save_model(DIRECTORY_NAME+'/BERT_model_final')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: forward() got an unexpected keyword argument 'labels'

In [ ]:
trainer.save_model(DIRECTORY_NAME)
tokenizer.save_pretrained(DIRECTORY_NAME)

## 4. Check the results